In [1]:
import csv
import numpy as np
import pandas as pd
from pandas import DataFrame
import sqlite3

from DATA225utils import make_connection, dataframe_query

In [2]:
conn_olap = make_connection(config_file = 'olap.ini')
cursor_olap = conn_olap.cursor()

## create olap tables 

In [ ]:
cursor_olap.execute(""" 
CREATE TABLE Calender_dm (
  date_id INT NOT NULL,
  Purchase_Date DATE NOT NULL,
  Day INT NOT NULL,
  Month INT NOT NULL,
  Quarter INT NOT NULL,
  Year INT NOT NULL,
  PRIMARY KEY (date_id)
);
""")


In [ ]:
cursor_olap.execute(""" 


CREATE TABLE Customer_dm (
  Customer_ID INT NOT NULL,
  Age INT NOT NULL,
  Gender ENUM('Male', 'Female', 'Other') NOT NULL,
  PreviousPurchases INT NOT NULL,
  Purchase_frequency VARCHAR(255) NOT NULL,
  PRIMARY KEY (Customer_ID)
);
""")

In [ ]:
cursor_olap.execute(""" 

CREATE TABLE Item_dm (
  Item_ID INT NOT NULL,
  ItemName VARCHAR(255) NOT NULL,
  Category VARCHAR(255) NOT NULL,
  PRIMARY KEY (Item_ID),
  Size VARCHAR(5) NOT NULL,
  Color VARCHAR(255) NOT NULL,
  Season VARCHAR(255) NOT NULL
);
""")



In [ ]:
cursor_olap.execute("""  CREATE TABLE Location_dm (
  Location_ID INT NOT NULL,
  Region VARCHAR(255) NOT NULL,
  State VARCHAR(255) NOT NULL,
  City VARCHAR(255) NOT NULL,
  PRIMARY KEY (Location_ID)
);
""")

In [ ]:
cursor_olap.execute(""" 
CREATE TABLE Payments_Shipping_dm (
  Payment_ID INT NOT NULL,
  PaymentMethod VARCHAR(255) NOT NULL,
  Shipping_type VARCHAR(255) NOT NULL,
  Discount_applied VARCHAR(255) NOT NULL,
  PRIMARY KEY (Payment_ID)
);
""")


In [ ]:


cursor_olap.execute(""" 
CREATE TABLE Reviews_Fct (
  ReviewRating DECIMAL(3, 2) CHECK (ReviewRating BETWEEN 1.00 AND 5.00) NOT NULL,
  Item_ID INT NOT NULL,
  Customer_ID INT NOT NULL,
  PRIMARY KEY (Item_ID, Customer_ID),
  CONSTRAINT Reviews_Fct_ibfk_1 FOREIGN KEY (Item_ID) REFERENCES Item_dm (Item_ID),
  CONSTRAINT Reviews_Fct_ibfk_2 FOREIGN KEY (Customer_ID) REFERENCES Customer_dm (Customer_ID)
);

""")


In [ ]:

cursor_olap.execute(""" 
CREATE TABLE Sales_Fct (
  Purchase_Amount DECIMAL(10, 2) NOT NULL,
  Customer_ID INT,
  Item_ID INT,
  Location_ID INT,
  Payment_ID INT,
  date_id int,
  PRIMARY KEY (Customer_ID, Item_ID, Location_ID, date_id, Payment_ID),
  KEY Sales_fct_ibfk_2 (Item_ID),
  KEY Sales_fct_ibfk_3 (Location_ID),
  KEY Sales_fct_ibfk_4 (date_id),
  KEY Sales_fct_ibfk_5 (Payment_ID),
  CONSTRAINT Sales_Fct_ibfk_1 FOREIGN KEY (Customer_ID) REFERENCES Customer_dm (Customer_ID),
  CONSTRAINT Sales_Fct_ibfk_2 FOREIGN KEY (Item_ID) REFERENCES Item_dm (Item_ID),
  CONSTRAINT Sales_Fct_ibfk_3 FOREIGN KEY (Location_ID) REFERENCES Location_dm (Location_ID),
  CONSTRAINT Sales_Fct_ibfk_4 FOREIGN KEY (date_id) REFERENCES Calender_dm (date_id),
  CONSTRAINT Sales_Fct_ibfk_5 FOREIGN KEY (Payment_ID) REFERENCES Payments_Shipping_dm (Payment_ID)
);
""")


In [ ]:
conn_olap.commit()

To describe how dimension and fact tables are populated from operational tables, let's assume we have two fact tables: `Sales_Fact` and `Customer_Activity_Fact`, along with dimension tables: `Customers_Dim`, `Items_Dim`, `Locations_Dim`, `Payments_Shipping_Dim`, and `Time_Dim`.

1. **Populating Dimension Tables:**

    - **Customers_Dim:**
      - Populate from the operational Customers Table.
      - Extract distinct values for each attribute (Age, Gender, Freq_Purchase).
      - Assign a surrogate key (Customer_SK) as the primary key.

    - **Items_Dim:**
      - Populate from the operational Items Table.
      - Extract distinct values for each attribute (Item_Name, Category, Size, Color).
      - Assign a surrogate key (Item_SK) as the primary key.

   
    - **Payements_Shipping_Dim:**
      - Populate from the operational PAYMENT AND SHIPPING Table.
      - Extract distinct values for the Shipping_Type attribute.
      - Assign a surrogate key (Shipping_SK) as the primary key.
      
      
    - **Locations_Dim:** (hierarchical)
      - Populate from the operational Locations Table.
      - Extract distinct values for each attribute (Region, State, City).
      - Assign a surrogate key (Location_SK) as the primary key.

   
    - **Calendr_Dim:** (hierarchical)
      - Populate from the operational Purchase Table with derived attributes.
      - Extract distinct values for the Date attribute.
      - Create additional attributes such as Year, Month, Day, Quarter, and Season.
      - Assign a surrogate key (Time_SK) as the primary key.

2. **Populating Fact Tables:**

    - **Sales_Fact:**
      - Populate from the operational Purchases Table.
      - Use foreign keys (Customer_SK, Item_SK, Location_SK, Shipping_SK, Promo_SK, Time_SK) obtained from the dimension tables.
      - Include measures such as Sales_Amount.

    - **Customer_Reviews_Fact:**
      - Populate from the operational Customers Table.
      - Use the foreign key (Customer_SK) obtained from the Customers_Dim table.
      - Include measures such as Customer_Reviwes, Freq_Of_Purchase.

In summary, the ETL (Extract, Transform, Load) process involves extracting data from the operational tables, transforming it into the required format for the dimension and fact tables, and loading it into the dimensional model. The surrogate keys in dimension tables serve as unique identifiers, and foreign keys in fact tables establish relationships with dimension tables. This dimensional model provides a structured and efficient way to analyze and query the data for business intelligence purposes.

## Load to Olap

In [3]:
cursor_olap.execute(""" 
INSERT INTO Calender_dm (date_id,Purchase_Date, Day, Month, Quarter, Year)
SELECT 
    Purchase_ID as date_id,
    Purchase_Date,
    DAY(Purchase_Date) AS Day,
    MONTH(Purchase_Date) AS Month,
    QUARTER(Purchase_Date) AS Quarter,
    YEAR(Purchase_Date) AS Year
FROM datadynamos_db.Purchases;
""")
conn_olap.commit()

In [6]:
cursor_olap.execute(""" INSERT INTO Customer_dm (Customer_ID, Age, Gender, PreviousPurchases, Purchase_frequency)
SELECT
    c.Customer_ID,
    c.Age,
    c.Gender,
    c.PreviousPurchases,
    p.Purchase_frequency
FROM datadynamos_db.Customers c
JOIN datadynamos_db.Purchases p ON c.Customer_ID = p.Customer_ID;
""")
conn_olap.commit()

In [11]:
cursor_olap.execute(""" INSERT INTO Item_dm (Item_ID, ItemName, Category, Size, Color, Season)
SELECT
    id.ItemDetails_ID as Item_ID,
    i.ItemName,
    i.Category,
    id.Size,
    id.Color,
    id.Season
FROM datadynamos_db.Items i
JOIN datadynamos_db.ItemDetails id ON i.Item_ID = id.Item_ID;
""")
conn_olap.commit()

In [14]:
cursor_olap.execute(""" INSERT INTO Location_dm (Location_ID, Region, State, City)
SELECT
    Location_ID,
    Region,
    State,
    City
FROM datadynamos_db.Location;
""")
conn_olap.commit()

In [16]:
cursor_olap.execute(""" INSERT INTO Payments_Shipping_dm (Payment_ID, PaymentMethod, Shipping_type, Discount_applied)
SELECT
    s.Shipping_ID as Payment_ID,
    p.PaymentMethod,
    s.Shipping_type,
    s.DiscountApplied
FROM datadynamos_db.Payment p
JOIN datadynamos_db.Shipping s ON p.Payment_ID = s.Payment_ID;
""")
conn_olap.commit()

In [17]:
cursor_olap.execute(""" INSERT INTO Reviews_Fct (ReviewRating, Item_ID, Customer_ID)
SELECT
    p.ReviewRating,
    i.Item_ID,
    c.Customer_ID
FROM datadynamos_db.Purchases p
JOIN Item_dm i ON p.Item_ID = i.Item_ID
JOIN Customer_dm c ON p.Customer_ID = c.Customer_ID;
""")
conn_olap.commit()

In [19]:
cursor_olap.execute(""" INSERT INTO Sales_Fct (Purchase_Amount, Customer_ID, Item_ID, Location_ID, date_id, Payment_ID)
SELECT
    p.Purchase_Amount,
    c.Customer_ID,
    i.Item_ID,
    l.Location_ID,
    cd.date_id,
    psd.Payment_ID
FROM datadynamos_db.Purchases p
JOIN Customer_dm c ON p.Customer_ID = c.Customer_ID
JOIN Item_dm i ON p.Item_ID = i.Item_ID
JOIN Location_dm l ON p.Location_ID = l.Location_ID
JOIN Calender_dm cd ON p.Purchase_Date = cd.Purchase_Date
JOIN Payments_Shipping_dm psd ON p.Payment_ID = psd.Payment_ID;
""")
conn_olap.commit()